In [ ]:
from pathlib import Path
import sys, os
import numpy as np
from pprint import pprint
from copy import deepcopy

# notebook이 examples/selectiveRIP에서 실행 중일 때
repo_root = Path.cwd().resolve().parents[1]  # .../qubit-experiment
sys.path.insert(0, str(repo_root))

In [ ]:

from laboneq.simple import *
from laboneq.dsl.device import DeviceSetup
from example_helpers.generate_descriptor import generate_descriptor


descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12256"],
    number_data_qubits=3,
    multiplex=True,
    number_multiplex=3,
    include_cr_lines=False,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="sibal_test",
    ip_address="192.168.0.83"
)




#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/QC6CH'


bus_ids = [f"b{i}" for i in range(3)]
bus_port = [4,5,6] #used 1,2,3 for qubit drive

for i, bus in zip(bus_port,bus_ids):
    setup.add_connections(
        setup.instruments[0].uid,
        # each bus uses its own drive:
        create_connection(
            to_signal=f"{bus}/drive",
            ports=f"SGCHANNELS/{i}/OUTPUT"
        ))

In [ ]:
setup.logical_signal_groups

# Initial QPU

In [ ]:
from qpu_types.fixed_transmon import FixedTransmonQubit, FixedTransmonQubitParameters
from qpu_types.bus_cavity import BusCavity, BusCavityParameters
from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform
from helper import load_qubit_parameters, save_qubit_parameters

qubit_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
bus_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("b")]

qubits = FixedTransmonQubit.from_device_setup(
    setup, qubit_uids=qubit_uids)
buses = BusCavity.from_device_setup(
    setup, qubit_uids=bus_uids)

qpu = QPU(quantum_elements={"qubits" : qubits, "bus" : buses}, quantum_operations=[FixedTransmonOperations, BusCavityOperations])


# from laboneq.simple import workflow
# folder_store = workflow.logbook.FolderStore("./experiment_store") 
# folder_store.activate()
# #folder_store.deactivate()
# #workflow.logbook.LoggingStore().activate()
# #workflow.logbook.LogbookStore().deactivate()


# SAVE QPU

From now on we save/load QPU elements not QuantumElements

In [ ]:
from laboneq.serializers import save, load, from_dict, from_json, to_dict, to_json
import time

t = time.localtime()
timestamp = time.strftime('%Y%m%d-%H%M', t)

filename = "save_test"
save(qpu, filename=f"./qpu_parameters/{timestamp}_{filename}")



# LOAD QPU

Likewise

In [ ]:
from datetime import datetime
def find_latest_json(folder_path):
    files = [f for f in os.listdir(folder_path)]
    timestamps = []
    for file in files:
        try:
            # Extract a timestamp assuming it's included in the filename
            timestamp_str = file.split('_', 1)[0]  # Assuming YYYYMMDDHHMMSS format
            timestamp = datetime.strptime(timestamp_str, '%Y%m%d-%H%M%S')
            timestamps.append((timestamp, file))
        except ValueError:
            print("fuck")
            continue  # Skip files that do not match the timestamp format

       # Find the most recent file
    if timestamps:
        latest_file = max(timestamps, key=lambda x: x[0])[1]
        return os.path.join(folder_path, latest_file)
    return None

qb_pars_file = find_latest_json("./qpu_parameters/")

from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
import laboneq.dsl.quantum.qpu as qpu_mod

class CombinedOperations(FixedTransmonOperations, BusCavityOperations):
    pass

qpu_mod.CombinedOperations = CombinedOperations

qpu = load(qb_pars_file)

buses = qpu.groups.bus
qubits = qpu.groups.qubits

# Connection

In [ ]:
session = Session(setup)
session.connect(do_emulation=True)

In [ ]:
from laboneq_applications.experiments import ramsey
options = ramsey.experiment_workflow.options()
# updates the qubit parameters "resonance_frequency_ge" and ge_T2_star
options.update(True)

transition_to_calibrate = "ge"
options.transition(transition_to_calibrate)
options.cal_states(transition_to_calibrate)

# Note: if neighbouring qubits are physically coupled by a resonator,
# you usually don't want to run Ramsey in parallel on them because
# your result will be skewed by strong residual-ZZ coupling.
# Next-nearest neighbours is typically okay.
qubits_to_measure = [qubits[0]]

delays = [
    np.linspace(0, 1e-6, 51)
    if transition_to_calibrate == "ef"
    else np.linspace(0, 20e-6, 51)                           
    for q in qubits_to_measure
]
detunings = [
    11.76e6 if transition_to_calibrate == "ef"
    else 0.673e6 for q in qubits_to_measure]

exp_workflow = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=qubits_to_measure,
    delays=delays,
    detunings=detunings,
    options=options
)

workflow_result = exp_workflow.run()

In [ ]:
qubits[0].parameters.ge_drive_pulse

In [ ]:
# Q2 q0 -> q0
qubits[0].parameters.drive_lo_frequency = 4.0e9
qubits[0].parameters.drive_range = 10
qubits[0].parameters.ge_drive_amplitude_pi = 0.6661475528508879
qubits[0].parameters.ge_drive_amplitude_pi2 = 0.33101490643459497
qubits[0].parameters.ge_drive_length = 64e-9

qubits[0].parameters.readout_amplitude = 0.5
qubits[0].parameters.readout_length = 1.2e-6
qubits[0].parameters.readout_integration_delay = 400e-9
qubits[0].parameters.readout_lo_frequency = 7.4e9
qubits[0].parameters.readout_pulse = {
            "function": "GaussianSquare",
            "sigma": 0.2,
            "risefall_sigma_ratio": 3.0
        }
qubits[0].parameters.readout_range_in = 0
qubits[0].parameters.readout_range_out = 0

qubits[0].parameters.resonance_frequency_ge = 4124420661.263548
qubits[0].parameters.readout_resonator_frequency = 7558567966.629468

# Q3 q1 -> q1




# Calibration

## Ramsey 

In [ ]:
from laboneq_applications.experiments import ramsey


q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temp_pars.resonance_frequency_ge = 4.9528e9
temp_pars.readout_length = 1.2e-6
temp_pars.readout_integration_length =1.2e-6
temporary_parameters[q.uid] = temp_pars
#######################################################################
delays = np.linspace(0,10e-6,301)
detunings = 0.6e6,
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(False)
options.count(512)
options.use_cal_traces(True)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options,
    temporary_parameters=temporary_parameters
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)